# Genre Classification using Support-vector Machine

In [0]:
import numpy as np
import librosa
import os
import matplotlib.pyplot as plt
import sklearn.svm
import IPython.display as ipd
import scipy as sp

In [0]:
!git clone https://github.com/Jakobovski/free-spoken-digit-dataset.git #directly clone the directory from github

# Feature computation

In [0]:
def compute_mfcc(audio, fs, n_mfcc):
    # Compute the spectrogram of the audio signal
    X = np.abs(librosa.stft(
        audio,
        window='hamming',
        n_fft=1024,
        hop_length=512,)
        )
    
    # Find the weights of the mel filters
    mel = librosa.filters.mel(
        sr=fs,
        n_fft=1024,
        n_mels=40,
        fmin=133.33,
        fmax=6853.8,
    )
    # Apply the filters to spectrogram
    melspectrogram = np.dot(mel, X)
    # Take the logarithm
    log_melspectrogram = np.log10(melspectrogram + 1e-16)
    
    # Apply the DCT to log melspectrogram to obtain the coefficients
    mfcc = sp.fftpack.dct(log_melspectrogram, axis=0, norm='ortho')[1:n_mfcc+1]
    return mfcc

### Compute training features

In [0]:
classes = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
test = []
train = []
n_mfcc = 13
dict_train_files = {'0': [], '1': [], '2': [], '3': [], '4': [], '5': [], '6': [], '7': [], '8': [], '9': []}
dict_train_features = {'0': [], '1': [], '2': [], '3': [], '4': [], '5': [], '6': [], '7': [], '8': [], '9': []}

train_root = 'free-spoken-digit-dataset/recordings'
class_train_files = [f for f in os.listdir(train_root) if f.endswith('.wav')] #outputs a list with the filename of every file in train_root

#Splits dataset in test and train
for f in class_train_files:
  first_split = f.rsplit("_", 1)[1]
  second_split = first_split.rsplit(".", 1)[0]
  if int(second_split) <= 4:
      test.append(f)
  else:
      train.append(f)

#Puts files name in dict_train_files according to class
for c in classes:
  for f in train:
    split = f.rsplit("_", 2)[0]
    if int(c) == int(split):
      dict_train_files[c].append(f)

#Computes feature matrix according to class and puts it into dict_train_features
for c in dict_train_files:
  
  train_features = np.zeros((len(dict_train_files[c]), n_mfcc)) # 
  
  for index, f in enumerate(dict_train_files[c]):
    audio, fs = librosa.load(os.path.join(train_root, f), sr=None)
    mfcc = compute_mfcc(audio, fs, n_mfcc)
    train_features[index, :] = np.mean(mfcc, axis=1)
    
    dict_train_features[c] = train_features

### Compute test features

In [0]:
#same as above for test set
dict_test_files = {'0': [], '1': [], '2': [], '3': [], '4': [], '5': [], '6': [], '7': [], '8': [], '9': []}
dict_test_features = {'0': [], '1': [], '2': [], '3': [], '4': [], '5': [], '6': [], '7': [], '8': [], '9': []}

for c in classes:
  for f in test:
    splitto = f.rsplit("_", 2)[0]
    if int(c) == int(splitto):
      dict_test_files[c].append(f)


for c in dict_test_files:
  
  n_test_samples = len(dict_test_files[c]) #outputs length of class_train_files 
  
  test_features = np.zeros((n_test_samples, n_mfcc))  
  
  for index, f in enumerate(dict_test_files[c]):
    audio, fs = librosa.load(os.path.join(train_root, f), sr=None)
    mfcc = compute_mfcc(audio, fs, n_mfcc)
    test_features[index, :] = np.mean(mfcc, axis=1)
    
  dict_test_features[c] = test_features

### Feature visualization

In [0]:
for c in classes:
    mfcc = dict_train_features[c].transpose()
    # Visualization
    fig = plt.figure(figsize=(16, 6))
    plt.subplot(1,2,1)
    plt.imshow(mfcc, origin='lower', aspect='auto')
    plt.xlabel('Training samples')
    plt.ylabel('MFCC coefficients')
    plt.title('MFCC (coefficients 0 to 13) for class {}'.format(c))
    plt.colorbar()
    plt.tight_layout()

    mfcc_upper = mfcc[4:]
    plt.subplot(1,2,2)
    plt.imshow(mfcc_upper, origin='lower', aspect='auto')
    plt.title('MFCC (coefficients 4 to 13) for class {}'.format(c))
    plt.xlabel('Training samples')
    plt.ylabel('MFCC coefficients')
    plt.colorbar()
    plt.tight_layout()

# SVM: multiclass case

In [0]:
class_0 = '0'
class_1 = '1'
class_2 = '2'
class_3 = '3'
class_4 = '4'
class_5 = '5'
class_6 = '6'
class_7 = '7'
class_8 = '8'
class_9 = '9'

#Training
X_train_0 = dict_train_features[class_0]
X_train_1 = dict_train_features[class_1]
X_train_2 = dict_train_features[class_2]
X_train_3 = dict_train_features[class_3]
X_train_4 = dict_train_features[class_4]
X_train_5 = dict_train_features[class_5]
X_train_6 = dict_train_features[class_6]
X_train_7 = dict_train_features[class_7]
X_train_8 = dict_train_features[class_8]
X_train_9 = dict_train_features[class_9]

y_train_0 = np.zeros((X_train_0.shape[0]))
y_train_1 = np.ones((X_train_1.shape[0]))
y_train_2 = np.full((X_train_2.shape[0]), 2)
y_train_3 = np.full((X_train_3.shape[0]), 3)
y_train_4 = np.full((X_train_4.shape[0]), 4)
y_train_5 = np.full((X_train_5.shape[0]), 5)
y_train_6 = np.full((X_train_6.shape[0]), 6)
y_train_7 = np.full((X_train_7.shape[0]), 7)
y_train_8 = np.full((X_train_8.shape[0]), 8)
y_train_9 = np.full((X_train_9.shape[0]), 9)

y_train_mc = np.concatenate((y_train_0, y_train_1, y_train_2, y_train_3, y_train_4, y_train_5, y_train_6, y_train_7, y_train_8, y_train_9), axis=0)

In [0]:
#Testing
X_test_0 = dict_test_features[class_0]
X_test_1 = dict_test_features[class_1]
X_test_2 = dict_test_features[class_2]
X_test_3 = dict_test_features[class_3]
X_test_4 = dict_test_features[class_4]
X_test_5 = dict_test_features[class_5]
X_test_6 = dict_test_features[class_6]
X_test_7 = dict_test_features[class_7]
X_test_8 = dict_test_features[class_8]
X_test_9 = dict_test_features[class_9]

y_test_0 = np.zeros((X_test_0.shape[0]))
y_test_1 = np.ones((X_test_1.shape[0]))
y_test_2 = np.full((X_test_2.shape[0]), 2)
y_test_3 = np.full((X_test_3.shape[0]), 3)
y_test_4 = np.full((X_test_4.shape[0]), 4)
y_test_5 = np.full((X_test_5.shape[0]), 5)
y_test_6 = np.full((X_test_6.shape[0]), 6)
y_test_7 = np.full((X_test_7.shape[0]), 7)
y_test_8 = np.full((X_test_8.shape[0]), 8)
y_test_9 = np.full((X_test_9.shape[0]), 9)

y_test_mc = np.concatenate((y_test_0, y_test_1, y_test_2, y_test_3, y_test_4, y_test_5, y_test_6, y_test_7, y_test_8, y_test_9), axis=0)

### Normalize features

In [0]:
feat_max = np.max(np.concatenate((X_train_0, X_train_1, X_train_2, X_train_3, X_train_4, X_train_5, X_train_6, X_train_7, X_train_8, X_train_9), axis=0), axis=0)
feat_min = np.min(np.concatenate((X_train_0, X_train_1, X_train_2, X_train_3, X_train_4, X_train_5, X_train_6, X_train_7, X_train_8, X_train_9), axis=0), axis=0)

X_train_0_normalized = (X_train_0 - feat_min) / (feat_max - feat_min)
X_train_1_normalized = (X_train_1 - feat_min) / (feat_max - feat_min)
X_train_2_normalized = (X_train_2 - feat_min) / (feat_max - feat_min)
X_train_3_normalized = (X_train_3 - feat_min) / (feat_max - feat_min)
X_train_4_normalized = (X_train_4 - feat_min) / (feat_max - feat_min)
X_train_5_normalized = (X_train_5 - feat_min) / (feat_max - feat_min)
X_train_6_normalized = (X_train_6 - feat_min) / (feat_max - feat_min)
X_train_7_normalized = (X_train_7 - feat_min) / (feat_max - feat_min)
X_train_8_normalized = (X_train_8 - feat_min) / (feat_max - feat_min)
X_train_9_normalized = (X_train_9 - feat_min) / (feat_max - feat_min)

X_train_mc_normalized = np.concatenate((X_train_0_normalized, X_train_1_normalized, X_train_2_normalized, X_train_3_normalized, X_train_4_normalized, X_train_5_normalized, X_train_6_normalized, X_train_7_normalized, X_train_8_normalized, X_train_9_normalized), axis=0)

X_test_0_normalized = (X_test_0 - feat_min) / (feat_max - feat_min)
X_test_1_normalized = (X_test_1 - feat_min) / (feat_max - feat_min)
X_test_2_normalized = (X_test_2 - feat_min) / (feat_max - feat_min)
X_test_3_normalized = (X_test_3 - feat_min) / (feat_max - feat_min)
X_test_4_normalized = (X_test_4 - feat_min) / (feat_max - feat_min)
X_test_5_normalized = (X_test_5 - feat_min) / (feat_max - feat_min)
X_test_6_normalized = (X_test_6 - feat_min) / (feat_max - feat_min)
X_test_7_normalized = (X_test_7 - feat_min) / (feat_max - feat_min)
X_test_8_normalized = (X_test_8 - feat_min) / (feat_max - feat_min)
X_test_9_normalized = (X_test_9 - feat_min) / (feat_max - feat_min)

X_test_mc_normalized = np.concatenate((X_test_0_normalized, X_test_1_normalized, X_test_2_normalized, X_test_3_normalized, X_test_4_normalized, X_test_5_normalized, X_test_6_normalized, X_test_7_normalized, X_test_8_normalized, X_test_9_normalized), axis=0)


### Define and train a model for each couple of classes

In [0]:
#Builds SVM classifier, play with the paramenters! look here: https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html 
SVM_parameters={
    'C': 1,
    'kernel': 'poly',
    'degree': 3,
    'decision_function_shape' : 'ovo',
    'probability' : True
}

clf = sklearn.svm.SVC(**SVM_parameters)


In [0]:
#fits the model
clf.fit(X_train_mc_normalized, y_train_mc)

SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovo', degree=3, gamma='scale', kernel='poly',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

### Evaluate each classifier

In [0]:
y_test_predicted = clf.predict(X_test_mc_normalized).reshape(-1, 1)

### Majority voting

In [0]:
y_test_predicted_mc = np.array(y_test_predicted, dtype=np.int)

In [0]:
y_test_predicted_mv = np.zeros((y_test_predicted_mc.shape[0],))
for i, e in enumerate(y_test_predicted_mc):
    y_test_predicted_mv[i] = np.bincount(e).argmax() 

### *Exercise 3* : Compute confusion matrix for multiclass
Compute and print the confusion matrix when more than 2 classes are presents.

In [0]:
def compute_cm_multiclass(gt, predicted):
    classes = np.unique(gt)
    
    CM = np.zeros((len(classes), len(classes)))
    
    for i in np.arange(len(classes)):
        pred_class = predicted[gt==i]
        
        for j in np.arange(len(pred_class)):
            CM[i, int(pred_class[j])] = CM[i, int(pred_class[j])] + 1 
    print(CM)
    

In [0]:
compute_cm_multiclass(y_test_mc, y_test_predicted_mv)

[[19.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0. 20.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. 20.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0. 18.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0. 20.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. 20.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. 19.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0. 20.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0. 20.  0.]
 [ 1.  1.  0.  0.  0.  1.  0.  0.  0. 17.]]
